In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data_train = pd.read_csv('../input/Google_Stock_Price_Train.csv')

data_train.tail()

In [ ]:
train = data_train.iloc[:,1:2].values

In [ ]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
train = mms.fit_transform(train)

In [ ]:
x_train = []
y_train = []

for i in range(60,1258):
    x_train.append(train[i-60:i,0])
    y_train.append(train[i,0])

In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
x_train.shape

In [ ]:
x_train

In [ ]:
x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM,CuDNNLSTM

In [ ]:
model = Sequential()
model.add(CuDNNLSTM(50,return_sequences=True,input_shape=(x_train.shape[1],1)))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(50,return_sequences=True))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(50,return_sequences=True))
model.add(Dropout(0.2))

model.add(CuDNNLSTM(50))
model.add(Dropout(0.2))

model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
model.fit(x_train,y_train,epochs=100,batch_size=32)

In [ ]:
data_test = pd.read_csv('../input/Google_Stock_Price_Test.csv')

data_test.head()

In [ ]:
real_values = data_test['Open'].values

In [ ]:
real_values

In [ ]:
data = pd.concat((data_train['Open'],data_test['Open']),axis=0)

In [ ]:
data

In [ ]:
inputs = data[len(data) - len(data_test) - 60:]

In [ ]:
inputs.iloc[59]

In [ ]:
inputs = data[len(data) - len(data_test) - 60:].values

In [ ]:
inputs.shape

In [ ]:
inputs = inputs.reshape(-1,1)

In [ ]:
inputs = mms.fit_transform(inputs)

In [ ]:
x_test = []

for i in range(60,80):
    x_test.append(inputs[i-60:i,0])

In [ ]:
x_test = np.array(x_test)
x_test = x_test.reshape(20,60,1)
x_test.shape

In [ ]:
prediction = model.predict(x_test)

In [ ]:
prediction = mms.inverse_transform(prediction)

In [ ]:
import matplotlib.pyplot as plt


plt.plot(prediction, color='blue', label='Predicted Values')
plt.plot(real_values, color='red', label='Real Values')
plt.legend()

plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error

mean_squared_error(real_values,prediction)